<a href="https://colab.research.google.com/github/James-R-Chapman/DTVariance/blob/main/DT_Variance_version4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfminer
!pip install pdfminer.six
!pip install fpdf

In [95]:
import io
import csv
import re
import pandas as pd
import pdfminer
import pdfminer.high_level
import pdfminer.layout
from google.colab import files
import os
from fpdf import FPDF
import numpy as np
from io import BytesIO

def extract_text_from_pdf(uploaded_file):
    # Save the uploaded PDF file to a BytesIO object
    pdf_data = io.BytesIO(uploaded_file)

    # Convert PDF to text
    txt_file_path = '/content/Report.txt'
    with open(txt_file_path, 'w') as output_file:
        output_file.write(pdfminer.high_level.extract_text(io.BytesIO(uploaded[pdf_file_name])))

    # Read the text from the file
    with open(txt_file_path, 'r') as f:
        text = f.read()

    # Find the index of the first line that contains "Part Number/Description"
    start_index = 0
    for i, line in enumerate(text.split('\n')):
        if "Part Number/Description" in line:
            start_index = i + 2  # Start 2 lines below the line containing "Part Number/Description"
            break

    # Split the text into lines
    lines = text.split('\n')

    # Open a new CSV file for writing
    csv_file_path = 'output.csv'
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)

        # Write the column headers
        writer.writerow(
            ['Page', 'Part Number', 'Description', 'Sheet', 'Y/P Bin', 'Sys Bin', 'Phy Count', 'Phy Y/P', 'Phy Tot',
             'Comp',
             'Var', 'Unit Cost', 'Dollar Variance', 'Open R/O', 'Total Value'])

        # Loop over the lines in the text
        current_page = 1  # Initialize current_page to 1
        for line in lines[start_index:]:
            # Extract the page number from the line using a regular expression
            match = re.search(r'Page\s+(\d+)', line)
            if match:
                current_page = match.group(1)
                continue

            # Extract the fields from the line based on their character count
            if ':' in line:
                parts = line.split(':', 1)
                part_num = parts[0].strip()
                if len(parts) > 1:
                    description = parts[1].strip()[:25]  # Use the first 40 characters of the description
                else:
                    description = ''
            else:
                part_num = ''
                description = ''

            fields = [
                current_page,
                part_num,
                description,
                line[43:48].strip(),  # Sheet
                line[48:60].strip(),  # Y/P Bin
                line[61:73].strip(),  # Sys Bin
                line[74:80].strip(),  # Phy Count
                line[80:86].strip(),  # Phy Y/P
                line[86:93].strip(),  # Phy Tot
                line[93:100].strip(),  # Comp
                line[100:107].strip(),  # Var
                line[107:118].strip(),  # Unit Cost
                line[118:130].strip(),  # Dollar Variance
                line[130:137].strip(),  # Open R/O
                line[138:].strip()  # Total Value
            ]

            # Write the fields to the CSV file
            writer.writerow(fields)

    return csv_file_path

# Upload the PDF file
uploaded = files.upload()

# Get the uploaded PDF file name
pdf_file_name = next(iter(uploaded))

# Extract text from PDF and convert to CSV
csv_file_path = extract_text_from_pdf(uploaded[pdf_file_name])

def filter_dataframe(df):
    # Define the regex pattern
    pattern = r'PD3350R'

    # Filter rows based on conditions
    df = df[~df['Part Number'].str.contains(pattern, na=False, regex=True)]
    df = df[~df['Sheet'].str.startswith(('='), na=False)]
    df = df[~df['Phy Count'].str.startswith('=', na=False)]
    df = df[~df['Total Value'].str.startswith('="', na=False)]

    # Additional conditions
    df = df[~((df['Sheet'].str.contains('Sheet', na=False)) &
              (df['Y/P Bin'].str.contains('Y/P Bi', na=False)) &
              (df['Sys Bin'].str.contains('Sys Bin', na=False)) &
              (df['Phy Count'].str.contains('Count', na=False)) &
              (df['Phy Y/P'].str.contains('Y/P', na=False)) &
              (df['Phy Tot'].str.contains('Tot', na=False)) &
              (df['Comp'].str.contains('Comp', na=False)) &
              (df['Var'].str.contains('Var', na=False)) &
              (df['Unit Cost'].str.contains('Cost', na=False)) &
              (df['Dollar Variance'].str.contains('Variance', na=False)) &
              (df['Open R/O'].str.contains('R/O', na=False)) &
              (df['Total Value'].str.contains('Value', na=False)))]

    # Add an additional condition
    df = df[~((df['Part Number'].isna()) &
              (df['Description'].isna()) &
              (df['Sheet'].isna()) &
              (df['Y/P Bin'].isna()) &
              (df['Sys Bin'].isna()) &
              (df['Phy Count'].isna()) &
              (df['Phy Y/P'].isna()) &
              (df['Phy Tot'].isna()) &
              (df['Comp'].isna()) &
              (df['Var'].isna()) &
              (df['Unit Cost'].isna()) &
              (df['Dollar Variance'].isna()) &
              (df['Open R/O'].isna()) &
              (df['Total Value'].isna()))]

    df = df[~((df['Part Number'].isna()) &
              (df['Description'].isna()) &
              (df['Sheet'].isna()) &
              (df['Y/P Bin'].isna()) &
              (df['Sys Bin'].isna()))]

    # Convert "Dollar Variance" column to numbers
    df['Dollar Variance'] = pd.to_numeric(df['Dollar Variance'], errors='coerce')

    return df

def generate_pdf(df, file_name):
    # Create a PDF object
    pdf = FPDF(orientation='L')

    # Set up document properties
    pdf.set_title('Variance Report')
    pdf.set_author('James Chapman')

    # Set up table header
    header = ['Page', 'Part Number', 'Description', 'Sheet', 'Y/P Bin', 'Sys Bin', 'Count', 'Phy Y/P', 'Phy Tot',
              'Comp', 'Var', 'Unit Cost', 'Variance', 'Open R/O', 'Total Value']
    pdf.set_font('Arial', 'B', 8)

    # Define the character counts for each column
    column_widths = [
        5,  # Page
        18,  # Part Number
        25,  # Description
        5,  # Sheet
        11,  # Y/P Bin
        11,  # Sys Bin
        7,  # Phy Count
        7,  # Phy Y/P
        7,  # Phy Tot
        7,  # Comp
        7,  # Var
        10,  # Unit Cost
        10,  # Dollar Variance
        10,  # Open R/O
        10   # Total Value
    ]

    row_height = 3.5
    # Calculate the scaled widths based on the character counts
    scaled_widths = [
        width * pdf.get_string_width('X')  # Convert character count to millimeters
        for width in column_widths
    ]

    # Set up table content
    pdf.set_font('Arial', '', 8)

    rows_per_page = 50  # Number of rows per page
    current_row = 0  # Current row count
    page_number = 1  # Starting page number

    # Calculate the total number of pages
    total_rows = len(df)
    total_pages = (total_rows - 1) // rows_per_page + 1

    for index, row in df.iterrows():
        # Check if a new page is needed
        if current_row % rows_per_page == 0:
            # Add a new page and print the header row
            pdf.add_page()
            for i, h in enumerate(header):
                # Left justify 'Part Number' and 'Description' columns
                if i == 1 or i == 2:
                    pdf.cell(scaled_widths[i], row_height, h, 1, 0, 'L')
                else:
                    pdf.cell(scaled_widths[i], row_height, h, 1, 0, 'C')
            pdf.ln(row_height)

            # Add page number at a specific coordinate
            pdf.set_font('Arial', '', 8)
            page_x = 275  # X-coordinate for page number
            page_y = 200  # Y-coordinate for page number
            pdf.text(page_x, page_y, f"Page {page_number} of {total_pages}")

            page_number += 1

        # Print row values
        for i, value in enumerate(row):
            if pd.isna(value):
                value = ''
            # Left justify 'Part Number' and 'Description' columns
            if i == 1 or i == 2:
                pdf.cell(scaled_widths[i], row_height, str(value), 1, 0, 'L')
            else:
                pdf.cell(scaled_widths[i], row_height, str(value), 1, 0, 'C')
        pdf.ln(row_height)

        current_row += 1

    # Save the PDF to a file
    pdf.output(file_name)

    return pdf, file_name


Saving Unit Variance Detail Final.PDF to Unit Variance Detail Final.PDF


In [96]:

# Read the CSV file
df = pd.read_csv(csv_file_path)

# Filter the DataFrame
df = filter_dataframe(df)

# Conditions for filtering rows
condition1 = (~df['Dollar Variance'].isna())
condition2 = (~df['Dollar Variance'].shift(1).isna() & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
condition3 = (~df['Dollar Variance'].shift(2).isna() & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
condition4 = (~df['Dollar Variance'].shift(3).isna() & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
condition5 = (~df['Dollar Variance'].shift(4).isna() & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
condition6 = (~df['Dollar Variance'].shift(5).isna() & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
condition7 = (df['Phy Tot'] == 'Not')

df_filtered = df[(condition1 | condition2 | condition3 | condition4 | condition5| condition6 | condition7)]

filtered_csv_file_path = '/content' + csv_file_path.replace('.CSV', '') + '_001.csv'
df_filtered.to_csv(filtered_csv_file_path, index=False)
# Generate PDF report for filtered data
#filtered_pdf_data = generate_pdf(df_filtered, "filtered_output.pdf")

# Save the filtered PDF report
filtered_pdf_file_path = '/content/' + pdf_file_name.replace('.PDF', '') + '_001.pdf'
filtered_pdf, _ = generate_pdf(df_filtered, filtered_pdf_file_path)

# Get the number of pages in the filtered PDF report
filtered_pdf_pages = filtered_pdf.page_no()
print(f"Filtered PDF file saved: {filtered_pdf_file_path}")
print(f"Number of Pages in PDF: {filtered_pdf_pages}")


Filtered PDF file saved: /content/Unit Variance Detail Final_001.pdf
Number of Pages in PDF: 77


In [98]:
# Additional set of rules
X = float(input("Enter the value of X: "))

# Read the CSV file again
df = pd.read_csv(csv_file_path)

df = filter_dataframe(df)

# Filter the DataFrame with different conditions
df_condition1 = (df['Dollar Variance'] >= X) | (df['Dollar Variance'] <= -X)
df_condition2 = (((df['Dollar Variance'].shift(1) >= X) | (df['Dollar Variance'].shift(1) <= -X)) & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
df_condition3 = (((df['Dollar Variance'].shift(2) >= X) | (df['Dollar Variance'].shift(2) <= -X)) & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
df_condition4 = (((df['Dollar Variance'].shift(3) >= X) | (df['Dollar Variance'].shift(3) <= -X)) & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
df_condition5 = (((df['Dollar Variance'].shift(4) >= X) | (df['Dollar Variance'].shift(4) <= -X)) & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
df_condition6 = (((df['Dollar Variance'].shift(5) >= X) | (df['Dollar Variance'].shift(5) <= -X)) & df['Part Number'].isna() & (~df['Y/P Bin'].isna() | ~df['Sys Bin'].isna()))
df_condition7 = (df['Phy Tot'] == 'Not')

df_conditional = df[(df_condition1 | df_condition2 | df_condition3 | df_condition4 | df_condition5 | df_condition6 | df_condition7)]

# Generate PDF report for conditional data
conditional_pdf_data = generate_pdf(df_conditional, "conditional_output.pdf")

# Save the conditional PDF report
conditional_pdf_file_path = '/content/' + pdf_file_name.replace('.PDF', '') + '_+-' + str(X) + '.pdf'
conditional_pdf, _ = generate_pdf(df_conditional, conditional_pdf_file_path)

# Get the number of pages in the conditional PDF report
conditional_pdf_pages = conditional_pdf.page_no()
print(f"Conditional PDF file saved: {conditional_pdf_file_path}")
print(f"Number of Pages in PDF: {conditional_pdf_pages}")

Enter the value of X: 200
Conditional PDF file saved: /content/Unit Variance Detail Final_+-200.0.pdf
Number of Pages in PDF: 20
